In [3]:
import os
os.chdir(r'C:\Users\a579330\Documents\AV')

import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np

from sklearn import preprocessing, model_selection
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import make_pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import f_regression
from sklearn.svm import SVC

from sklearn.decomposition import PCA
from sklearn.metrics import roc_curve, auc, classification_report, confusion_matrix

from imblearn.under_sampling import RandomUnderSampler, NearMiss, CondensedNearestNeighbour
from imblearn.under_sampling import EditedNearestNeighbours, RepeatedEditedNearestNeighbours, TomekLinks
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.ensemble import BalanceCascade, EasyEnsemble
from sklearn.ensemble import AdaBoostClassifier
#try class weight sampling 

from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

import seaborn as sns
sns.set(color_codes = True)

In [4]:
df = pd.read_csv('train_ajEneEa.csv')

In [5]:
df.smoking_status.unique()

array([nan, 'never smoked', 'formerly smoked', 'smokes'], dtype=object)

In [6]:
df.shape

(43400, 12)

In [7]:
df.describe()

,id,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke
count,43400.000000,43400.000000,43400.000000,43400.000000,43400.000000,41938.000000,43400.000000
mean,36326.142350,42.217894,0.093571,0.047512,104.482750,28.605038,0.018041
std,21072.134879,22.519649,0.291235,0.212733,43.111751,7.770020,0.133103
min,1.000000,0.080000,0.000000,0.000000,55.000000,10.100000,0.000000
25%,18038.500000,24.000000,0.000000,0.000000,77.540000,23.200000,0.000000
50%,36351.500000,44.000000,0.000000,0.000000,91.580000,27.700000,0.000000
75%,54514.250000,60.000000,0.000000,0.000000,112.070000,32.900000,0.000000
max,72943.000000,82.000000,1.000000,1.000000,291.050000,97.600000,1.000000


In [8]:
df.stroke.sum()
#Event rate is only 1.8%

783

In [9]:
df.isnull().sum()

id                       0
gender                   0
age                      0
hypertension             0
heart_disease            0
ever_married             0
work_type                0
Residence_type           0
avg_glucose_level        0
bmi                   1462
smoking_status       13292
stroke                   0
dtype: int64

In [10]:
df['bmi'] = df.bmi.replace(np.NaN, 0)
df['bmi'] = df['bmi'].replace(0, df['bmi'].mean())

In [11]:
df['smoking_status'] = df['smoking_status'].replace(np.NaN, 'NotAvailable')

In [12]:
#encode the smoking status column
'''
df['smoking_status'] = df['smoking_status'].replace(np.NaN, '')
le = LabelEncoder()
le.fit(df['smoking_status'])
df['smoking_status'] = list(le.transform(df['smoking_status'].astype(str)))

#encode the Residence_type column
le1 = LabelEncoder()
le1.fit(df['Residence_type'])
df['Residence_type'] = list(le1.transform(df['Residence_type']))

#encode the work_type column
le2 = LabelEncoder()
le2.fit(df['work_type'])
df['work_type'] = list(le2.transform(df['work_type']))

#encode the ever_married column
le3 = LabelEncoder()
le3.fit(df['ever_married'])
df['ever_married'] = list(le3.transform(df['ever_married']))

#encode the ever_married column
le4 = LabelEncoder()
le4.fit(df['gender'])
df['gender'] = list(le4.transform(df['gender']))
'''

"\ndf['smoking_status'] = df['smoking_status'].replace(np.NaN, '')\nle = LabelEncoder()\nle.fit(df['smoking_status'])\ndf['smoking_status'] = list(le.transform(df['smoking_status'].astype(str)))\n\n#encode the Residence_type column\nle1 = LabelEncoder()\nle1.fit(df['Residence_type'])\ndf['Residence_type'] = list(le1.transform(df['Residence_type']))\n\n#encode the work_type column\nle2 = LabelEncoder()\nle2.fit(df['work_type'])\ndf['work_type'] = list(le2.transform(df['work_type']))\n\n#encode the ever_married column\nle3 = LabelEncoder()\nle3.fit(df['ever_married'])\ndf['ever_married'] = list(le3.transform(df['ever_married']))\n\n#encode the ever_married column\nle4 = LabelEncoder()\nle4.fit(df['gender'])\ndf['gender'] = list(le4.transform(df['gender']))\n"

In [13]:
df = pd.get_dummies(df)

In [14]:
df

,id,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,gender_Female,gender_Male,gender_Other,ever_married_No,ever_married_Yes,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_NotAvailable,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,30669,3.0,0,0,95.12,18.000000,0,0,1,0,1,0,0,0,0,0,1,1,0,1,0,0,0
1,30468,58.0,1,0,87.96,39.200000,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0
2,16523,8.0,0,0,110.89,17.600000,0,1,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0
3,56543,70.0,0,0,69.04,35.900000,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0
4,46136,14.0,0,0,161.28,19.100000,0,0,1,0,1,0,0,1,0,0,0,1,0,1,0,0,0
5,32257,47.0,0,0,210.95,50.100000,0,1,0,0,0,1,0,0,1,0,0,0,1,1,0,0,0
6,52800,52.0,0,0,77.59,17.700000,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0
7,41413,75.0,0,1,243.53,27.000000,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0
8,15266,32.0,0,0,77.67,32.300000,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1
9,28674,74.0,1,0,205.84,54.600000,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0


In [15]:
y = df.stroke
X = df.drop(["id", "stroke"], axis = 1)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, random_state = 0)

C:\Users\a579330\AppData\Local\Continuum\anaconda2\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [17]:
#splitting up the training dataset 1 and 0 based of y train
X_1, X_2 = X_train[y_train == 1].as_matrix(), X_train[y_train == 0].as_matrix()

In [18]:
pipelines = { 
    'l2' : make_pipeline(StandardScaler(), LogisticRegression(penalty='l2' , random_state = 123)),
    'rf' : make_pipeline(StandardScaler(), RandomForestClassifier(random_state = 123)),
    'gb' : make_pipeline(StandardScaler(), GradientBoostingClassifier(random_state = 123)),
    'dt' : make_pipeline(StandardScaler(), DecisionTreeClassifier(random_state=123)),
    'svc': make_pipeline(StandardScaler(), SVC(random_state=123, probability=True))
}

In [19]:
l2_hyperparameters = {
    'logisticregression__C' : np.linspace(1e-3, 1e3, 10), #0.001 TO 1000
}

rf_hyperparameters = {
    'randomforestclassifier__n_estimators': [100, 150, 200],
    'randomforestclassifier__max_features': ['auto', 'sqrt', 0.33]
}

gb_hyperparameters = {
    'gradientboostingclassifier__n_estimators': [100, 150, 200],
    'gradientboostingclassifier__learning_rate': [0.05, 0.1, 0.2],
    'gradientboostingclassifier__max_depth': [1, 3, 5]
}

dt_hyperparameters = {
    'decisiontreeclassifier__criterion': ["gini", "entropy"],
    'decisiontreeclassifier__min_samples_split': [2, 10, 20],
    'decisiontreeclassifier__max_depth': [None, 2, 5, 10],
    'decisiontreeclassifier__min_samples_leaf': [1, 5, 10],
    'decisiontreeclassifier__max_leaf_nodes': [None, 5, 10, 20]
}

svc_hyperparameters = {
    'svc__C': [0.001, 0.01, 0.1, 1, 10],
    'svc__gamma': [0.001, 0.01, 0.1, 1],
    'svc__kernel': ['linear', 'poly', 'rbf', 'sigmoid']     
}

In [20]:
hyperparameters = {
    'l2' : l2_hyperparameters,
    'rf' : rf_hyperparameters,
    'gb' : gb_hyperparameters,
    'dt' : dt_hyperparameters,
    'svc': svc_hyperparameters
}

#random undersampler
us = RandomUnderSampler(ratio = 0.5, random_state=1)
X_train_res, y_train_res = us.fit_sample(X_train,y_train)

#near miss - 1: Retain points from majority class whose mean distance to the k-nearest points in minority is lowest
us_nm1 = NearMiss(ratio=0.5, size_ngh = 3, version = 1, random_state=1)
X_train_res, y_train_res = us_nm1.fit_sample(X_train,y_train)
#lead to information loss

#near miss - 2: Retain points from majority class whose mean distance to the k-nearest points in minority is farthest
us_nm2 = NearMiss(ratio=0.5, size_ngh = 3, version = 2, random_state=1)
X_train_res, y_train_res = us_nm2.fit_sample(X_train,y_train)
#same as near miss 1. leads to information loss
#preserves all the points that are close to all points of the minority class

#near miss - 3: Take k nearest neighbours of everypoint in the minority class
us_nm3 = NearMiss(ratio=0.5, size_ngh = 3, ver3_samp_ngh=3, version = 3, random_state=1)
X_train_res, y_train_res = us_nm3.fit_sample(X_train,y_train)

#Condensed nearest neighbour: is a data reduction technique for doing knn. it selects the set of prototypes U from the training data, such that 1NN with U can classify the examples almost as accurately as 1NN does with the whole data set.
us_cnn = CondensedNearestNeighbour(random_state=1)
X_train_res, y_train_res = us_cnn.fit_sample(X_train,y_train)
#worse than all resampling techniques removes examples whose class differs from majority of its nearest neighbours

#ENN = edited nearest neighbours: this removes examples whose class differs from majority of its nearest neighbours
us_enn = EditedNearestNeighbours(size_ngh=5, random_state=1)
X_train_res, y_train_res = us_enn.fit_sample(X_train,y_train)

#random-oversampler
os = RandomOverSampler(ratio = 0.5, random_state=1)
X_train_res, y_train_res = os.fit_sample(X_train,y_train)

#SMOTE: Synthetic minority over sampling technique
#for each p in minority class
#Compute k nearest neighbors in minority
#2. Randomly chose r elements <= k, of the neighbors (with replacement)
#3. chose a random point along each of the lines joining p and each of the r neighbors
#4. add these synthetic points to the dataset with minority class
os_smote = SMOTE(ratio = 0.5, k = 5, random_state=1)
X_train_res, y_train_res = os_smote.fit_sample(X_train,y_train)

os_smoteTomek = SMOTETomek(ratio = 0.5, k = 5, random_state=1)
X_train_res, y_train_res = os_smoteTomek.fit_sample(X_train,y_train)

os_smoteENN = SMOTEENN(ratio = 0.5, k = 5,size_ngh = 5, random_state=1)
X_train_res, y_train_res = os_smoteENN.fit_sample(X_train,y_train)

In [21]:
os_smoteENN = SMOTEENN(ratio = 0.5, k = 5,size_ngh = 5, random_state=1)
X_train_res, y_train_res = os_smoteENN.fit_sample(X_train,y_train)

C:\Users\a579330\AppData\Local\Continuum\anaconda2\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\a579330\AppData\Local\Continuum\anaconda2\lib\site-packages\imbalanced_learn-0.3.3-py2.7.egg\imblearn\utils\deprecation.py:50: DeprecationWarning: 'k' is deprecated from 0.2 and will be removed in 0.4. Use 'k_neighbors' instead.
  category=DeprecationWarning)
C:\Users\a579330\AppData\Local\Continuum\anaconda2\lib\site-packages\imbalanced_learn-0.3.3-py2.7.egg\imblearn\utils\deprecation.py:50: DeprecationWarning: 'm' is deprecated from 0.2 and will be removed in 0.4. Use 'm_neighbors' instead.
  category=DeprecationWarning)
C:\Users\a579330\AppData\Local\Continuum\anaconda2\lib\site-packages\imbalanced_learn-0.3.3-py2.7.egg\imblearn\utils\depr

In [ ]:
#Create empty dictionary called fitted_models
fitted_models = {}

# Loop through model pipelines, tuning each one and saving it to fitted_models
for name, pipeline in pipelines.items():
    # Create cross-validation object from pipeline and hyperparameters
    # number of jobs to run in parallel
    model = GridSearchCV(pipeline, hyperparameters[name], cv=10, n_jobs=-1)
    
    print "Fitting ", name
    print "------------------"
    # Fit model on X_train, y_train
    model.fit(X_train_res, y_train_res)
    
    #Printing best parameters
    print ("Best parameters set of development set:")
    print (model.best_params_)
    print "__________________________________________"
        
    '''print ("Grid scores on development set:")
    means = model.cv_results_['mean_test_score']
    stds = model.cv_results_['std_test_score']
        
    for mean, std, params in zip(means, stds, model.cv_results_['params']):
        #mean score and 95% confidence interval
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    print "__________________________________________"'''
    
    print ("\nBest Estimators:")
    print (model.best_estimator_, "\n")   
    
    
    # Store model in fitted_models[name] 
    fitted_models[name] = model
    
    # Print '{name} has been fitted'
    print "----------------------------"
    print(name, 'has been fitted.\n')
    print "----------------------------"

Fitting  rf
------------------
Best parameters set of development set:
{'randomforestclassifier__n_estimators': 150, 'randomforestclassifier__max_features': 0.33}
__________________________________________

Best Estimators:
(Pipeline(memory=None,
     steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=0.33, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_i...imators=150, n_jobs=1,
            oob_score=False, random_state=123, verbose=0, warm_start=False))]), '\n')
----------------------------
('rf', 'has been fitted.\n')
----------------------------
Fitting  l2
------------------
Best parameters set of development set:
{'logisticregression__C': 111.11200000000001}
__________________________________________

Best Estimators:
(Pipeline(memory=None,
     steps=[('standardscaler', StandardScaler(co

In [ ]:
print "Holdout accuracy scores (Percentage of observations correctly classified) \n"

for name, model in fitted_models.items():
    print name, ": ", model.best_score_ 

In [203]:
# Predict classes using L1-regularized logistic regression 
pred = fitted_models['knn'].predict_proba(X_test)
pred = [p[1] for p in pred]
# Display first 10 predictions
pred[:10]

#confusionMatrix = pd.DataFrame(confusion_matrix(y_test, pred).tolist(), columns = ['Predicted: No', 'Predicted: Yes'])
#confusionMatrix

[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [205]:
print classification_report(y_test, fitted_models['knn'].predict(X_test))

             precision    recall  f1-score   support

          0       0.99      0.96      0.97     12800
          1       0.07      0.16      0.09       221

avg / total       0.97      0.95      0.96     13021



In [165]:
len(pred)

13021

In [166]:
len(y_test)

13021

In [206]:
fpr, tpr, thresholds = roc_curve(y_true=y_test, y_score=pred)

In [207]:
pd.DataFrame({'FPR': fpr, 'TPR' : tpr, 'Thresholds' : thresholds})

,FPR,TPR,Thresholds
0,0.000000,0.000000,2.0
1,0.040391,0.162896,1.0
2,1.000000,1.000000,0.0


In [ ]:

# Initialize figure
fig = plt.figure(figsize=(8,8))
plt.title('Receiver Operating Characteristic')

# Plot ROC curve
plt.plot(fpr, tpr, label='l1')
plt.legend(loc='lower right')

# Diagonal 45 degree line
plt.plot([0,1],[0,1],'k--')

# Axes limits and labels
plt.xlim([-0.1,1.1])
plt.ylim([-0.1,1.1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [181]:
print("AUC from hold out set:", auc(fpr, tpr) )

('AUC from hold out set:', 0.8577407381221721)
